# Summary

# Imports

In [3]:
%run _imports.ipynb

Setting the PYTHON_VERSION environment variable.
Setting the SPARK_MASTER environment variable.
Setting the DB_TYPE environment variable.
Setting the DB_PORT environment variable.


2017-12-02 19:33:51.030316


In [4]:
NOTEBOOK_NAME = 'pmc_statistics'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

In [ ]:
# Articles

# PMC suppl

#### pmc_suppl_files

In [7]:
import pmc_tables.download

In [8]:
ftp = pmc_tables.download.get_ftp_client('ebi')

In [9]:
pmc_suppl_files = []

for subset in ['NON-OA', 'OA']:
    subset_folders = ftp.nlst(f'/pub/databases/pmc/suppl/{subset}/')
    for folder in tqdm.tqdm_notebook(subset_folders, total=len(subset_folders)):
        for pmc_file in ftp.nlst(folder):
            pmc_suppl_files.append(pmc_file)

HBox(children=(IntProgress(value=0, max=1273), HTML(value='')))

HBox(children=(IntProgress(value=0, max=1208), HTML(value='')))

In [10]:
pmc_suppl_files[:10]

['/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102254.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102269.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102275.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102277.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102329.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102330.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102388.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102394.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102396.zip',
 '/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102408.zip']

In [11]:
with open(f"{NOTEBOOK_NAME}/pmc_suppl_files.pickle", 'wb') as fout:
    pickle.dump(pmc_suppl_files, fout, pickle.HIGHEST_PROTOCOL)

#### ebi_pmc_suppl

In [12]:
ebi_pmc_suppl = [op.basename(f).partition('.')[0] for f in pmc_suppl_files]

In [13]:
len(ebi_pmc_suppl)

1623854

In [14]:
ebi_pmc_suppl = set(ebi_pmc_suppl)

In [15]:
len(ebi_pmc_suppl)

1623854

In [17]:
ebi_pmc_suppl_non_oa = {op.basename(f).partition('.')[0] for f in pmc_suppl_files if '/NON-OA/' in f}

In [18]:
ebi_pmc_suppl_oa = {op.basename(f).partition('.')[0] for f in pmc_suppl_files if '/OA/' in f}

In [16]:
with open(f"{NOTEBOOK_NAME}/ebi_pmc_suppl.pickle", 'wb') as fout:
    pickle.dump(ebi_pmc_suppl, fout, pickle.HIGHEST_PROTOCOL)

In [19]:
with open(f"{NOTEBOOK_NAME}/ebi_pmc_suppl_oa.pickle", 'wb') as fout:
    pickle.dump(ebi_pmc_suppl_oa, fout, pickle.HIGHEST_PROTOCOL)t

In [ ]:
with open(f"{NOTEBOOK_NAME}/ebi_pmc_suppl_non_oa.pickle", 'wb') as fout:
    pickle.dump(ebi_pmc_suppl_non_oa, fout, pickle.HIGHEST_PROTOCOL)

#### ebi_suppl_df

In [35]:
df = pd.DataFrame(pmc_suppl_files, columns=['File'])

In [36]:
df.head()

,File
0,/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102254.zip
1,/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102269.zip
2,/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102275.zip
3,/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102277.zip
4,/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102329.zip


In [52]:
df['pmc_id'] = df['File'].apply(lambda f: op.basename(f).partition('.')[0])
df['source_url'] = 'ftp://ftp.ebi.ac.uk' + df['File']
df['output_dir'] = df['File'].str[19:].apply(op.dirname) + '/' + df['pmc_id']

In [53]:
ebi_suppl_df = df[['pmc_id', 'source_url', 'output_dir']]

In [54]:
ebi_suppl_df.head()

,pmc_id,source_url,output_dir
0,PMC102254,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102254.zip,suppl/NON-OA/PMC101900-PMC105899/PMC102254
1,PMC102269,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102269.zip,suppl/NON-OA/PMC101900-PMC105899/PMC102269
2,PMC102275,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102275.zip,suppl/NON-OA/PMC101900-PMC105899/PMC102275
3,PMC102277,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102277.zip,suppl/NON-OA/PMC101900-PMC105899/PMC102277
4,PMC102329,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/NON-OA/PMC101900-PMC105899/PMC102329.zip,suppl/NON-OA/PMC101900-PMC105899/PMC102329


In [56]:
ebi_suppl_df.to_csv(f'{NOTEBOOK_NAME}/ebi_suppl_df.csv', index=False)

In [57]:
assert not set(ebi_suppl_df['pmc_id']) ^ ebi_pmc_suppl